<a href="https://colab.research.google.com/github/protogia/formula1-evaluations/blob/main/formula1-gp-brazil-preview-2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prologue
It's 5 days before the GP Brazil is starting the next session of Formula1 season 2025 and I decided to evaluate the last 2024 race of this unique circuit with all its special characteristics to make a little preview.

Moreover this notebook is also a try to get some inspiration for a greater analysis of multiple seasons. I am planning to compare as good as possible historical data of choosen cirquits. Therefore I need to get a feeling what kind of analysis makes sense or not.

But for now let's focus on the preview of 2025. The GP Brazil will be held on the Interlagos Race Track. The drivers will complete 71 laps on the 4.309km long circuit and typically have to contend with harsh weather conditions as we will see in the further analysis.


## Preparing

In the next steps we'll install necessary packages, do some preconfigurations and load the data using _fastf1_.

### Install fastf1


In [ ]:
%%capture
!pip install fastf1;

import fastf1

### Preconfiguration

In [ ]:
# log-config
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# layout-config
from IPython.core.display import display, HTML
display(HTML(""))

In [ ]:
# data-config
fastf1.Cache.enable_cache('/content')

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Loading and Preparing Data

In [ ]:
race = fastf1.get_session(2024, "Brazil", identifier="R")
race.load(telemetry=True)

INFO:fastf1.fastf1.core:Loading data for São Paulo Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
INFO:fastf1.api:Fetching session info data...
INFO:fastf1.fastf1.req:Data has been written to cache!
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
INFO:fastf1.api:Fetching driver list...
INFO:fastf1.fastf1.req:Data has been written to cache!
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
INFO:fastf1.api:Fetching session status data...
INFO:fastf1.fastf1.req:Data has been written to cache!
INFO:fastf1.fastf1.req:No cached data found for lap_count. Loading data...
INFO:fastf1.api:Fetching lap count data...
INFO:fastf1.fastf1.req:Data has been written to cache!
INFO:fastf1.fastf1.req:No cached data found for track_status_data. Loading data...
INFO:fastf1.api:Fetching track status data...
INFO:fastf1.fastf1.req:Data has been written to cache!
INFO:fastf1.fastf1.req:No 

## Track Overview
As mentioned the circuit in Sao Paulo has a length of 4.309km seperated into 15 sectors/corners. It contains two highspeed sections as well as small corners with different radius.

As shown in the next chart especially the section from _corner 3_ up to _corner 7_ as well as _corner 8_ to _corner 12_ are characterized by small radiuses and at the same time by many changes of the gradient.

The long high speed sections are also different to each other. While the section from _corner 13_ to _corner 15_ changes the gradient four times, the second section (_sector 3_) has a negative steep hill downwards with a gradient up to -8.75% which makes it hard for the drivers to find the right brake point when entering _corner 4_.

In [ ]:
position = race.laps.pick_fastest().get_pos_data()
circuit_info = race.get_circuit_info()

In [ ]:
def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

In [ ]:
# Get an array of shape [n, 2] where n is the number of points and the second
# axis is x and y.
track = position.loc[:, ('X', 'Y')].to_numpy()

# Convert the rotation angle from degrees to radian.
track_angle = circuit_info.rotation / 180 * np.pi

# Rotate and plot the track map.
rotated_track = rotate(track, angle=track_angle)

In [ ]:
reference_altitude = 800

# Assuming the Z data is already in meters, we just need to add the reference altitude
altitude_meters = position['Z'].values + reference_altitude

# Calculate the gradient of the altitude
# Using numpy.gradient to calculate the gradient along the track points
# We need to calculate the gradient with respect to distance along the track, not just the index
# A simplified approach is to calculate the difference between consecutive altitude values
altitude_gradient = np.gradient(altitude_meters)


# scatter plot with color scale based on the altitude gradient
fig = go.Figure(data=go.Scatter(
    x=rotated_track[:, 0],
    y=rotated_track[:, 1],
    mode='lines+markers',
    marker=dict(
        size=5,
        color=altitude_gradient,
        colorscale='Plasma',
        colorbar=dict(title='Altitude Gradient'),
        opacity=0.8
    ),
    line=dict( # track
        color='grey',
        width=1
    ),
    hoverinfo='text',
    text=[f'Altitude Gradient: {grad:.2f}%' for grad in altitude_gradient] # Set the hover text
))

# add corner information as annotations
track_angle = circuit_info.rotation / 180 * np.pi # track rotation angle

for _, corner in circuit_info.corners.iterrows():
    # Create a string from corner number and letter
    txt = f"{corner['Number']}{corner['Letter']}"

    # Rotate the center of the corner equivalently to the rest of the track map
    track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_angle)

    # Add annotation for the corner number directly at the corner's rotated coordinates
    fig.add_annotation(
        x=track_x,
        y=track_y,
        text=txt,
        showarrow=False, # Do not show arrow
        bgcolor="grey",
        font=dict(
            color="white",
            size=10
        )
    )

fig.update_layout(
    title='Track Overview with Altitude Gradient and Corners',
    xaxis_title='X Coordinate',
    yaxis_title='Y Coordinate',
    yaxis=dict(scaleanchor="x", scaleratio=1), # Ensure aspect ratio is equal
)

fig.show()

The following lineplot shows the altitude gradient over all corners to make this special point clearer. If we can trust the data there is a lot of changes in altitude even between the short sections between the corners.

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.colors as colors # Import plotly.colors

reference_altitude = 800

# Assuming the Z data is already in meters, we just need to add the reference altitude
altitude_meters = position['Z'].values + reference_altitude

# Calculate the gradient of the altitude
# Using numpy.gradient to calculate the gradient along the track points
altitude_gradient = np.gradient(altitude_meters)

# Calculate the distance along the track
# Calculate the difference in X and Y between consecutive points
delta_x = position['X'].diff().fillna(0)
delta_y = position['Y'].diff().fillna(0)

# Calculate the distance between consecutive points
distances = np.sqrt(delta_x**2 + delta_y**2)

# Calculate the cumulative distance along the track
cumulative_distance = distances.cumsum()/10


# Create a color scale based on the altitude gradient values
colorscale = 'Plasma'
min_gradient, max_gradient = np.min(altitude_gradient), np.max(altitude_gradient)

# Get the colors from the colorscale
plasma_colors = colors.get_colorscale(colorscale)

# Create a list of segments with start and end points and their corresponding gradient and color
segments = []
for i in range(len(altitude_gradient) - 1):
    segment_gradient = (altitude_gradient[i] + altitude_gradient[i+1]) / 2 # Average gradient for the segment
    normalized_segment_gradient = (segment_gradient - min_gradient) / (max_gradient - min_gradient) if (max_gradient - min_gradient) != 0 else 0

    # Interpolate color from the colorscale
    segment_color = colors.sample_colorscale(plasma_colors, normalized_segment_gradient)[0]


    segment = {
        'x': [cumulative_distance.iloc[i], cumulative_distance.iloc[i+1]], # Use cumulative distance for x
        'y': [altitude_gradient[i], altitude_gradient[i+1]],
        'gradient': segment_gradient,
        'color': segment_color # Store the calculated color for the segment
    }
    segments.append(segment)

# Create the figure
fig = go.Figure()

# Add each segment as a separate Scatter trace with a colored line
for segment in segments:
    fig.add_trace(go.Scatter(
        x=segment['x'],
        y=segment['y'],
        mode='lines', # Only lines, no markers needed for this visualization
        line=dict(color=segment['color'], width=2), # Color the line by segment gradient
        hoverinfo='text',
        text=f'Altitude Gradient: {segment["gradient"]:.2f}',
        showlegend=False # Hide legend for individual segments
    ))

# Add a single Scatter trace for the colorbar. We use the original data for this.
fig.add_trace(go.Scatter(
    x=[None], # No x or y data
    y=[None],
    mode='markers', # Use markers mode to display the colorbar
    marker=dict(
        colorscale=colorscale,
        showscale=True,
        colorbar=dict(title='Altitude Gradient'),
        cmin=min_gradient,
        cmax=max_gradient,
        color=altitude_gradient # Use the full gradient data for the color mapping in the colorbar trace
    ),
    hoverinfo='none',
    showlegend=False
))


# Add vertical lines for corner information
for _, corner in circuit_info.corners.iterrows():
    # Find the cumulative distance at the corner's position
    # This requires finding the point in the cumulative_distance Series closest to the corner's X and Y coordinates.
    # We can approximate this by finding the index in the position data closest to the corner's position
    distances_to_corner = np.sqrt((position['X'] - corner['X'])**2 + (position['Y'] - corner['Y'])**2)
    closest_pos_index = distances_to_corner.idxmin()
    corner_cumulative_distance = cumulative_distance.iloc[closest_pos_index]


    # Add a vertical line at the closest cumulative distance
    fig.add_vline(
        x=corner_cumulative_distance,
        line_width=1,
        line_dash="dash",
        line_color="red",
        annotation_text=f"C-{corner['Number']}{corner['Letter']}",
        annotation_position="top right"
    )


fig.update_layout(
    title='Altitude Gradient Along the Track with Corners',
    xaxis_title='Distance along Track [m]', # Update x-axis title
    yaxis_title='Altitude Gradient [%]',
)

fig.show()

As if the track weren't challenging enough, the GP Brazil is ​​also known for difficult weather conditions. Sao Paulo is characterized by subtropical climate conditions and november is typically the start of summer there. [This leads into an average amount of precipitation of 145l/m² or in simple words: It's raining a lot. Furthermore the average temperature lays between 15,6°C and 24,9°C in this period while the average relative humidity is around 73.7%](https://en.wikipedia.org/wiki/S%C3%A3o_Paulo#cite_ref-NCB-1931-1960_83-0).

The next chart shows the weather conditions for the GP Brazil 2024. Almost half of the race was driven while raining. The temperature was between 23°C at the beginning of the race and 20°C at the end, whereas the track temperature layed between 29,5°C at driest phase of the race and 23,3°C when it was raining.

In [ ]:
from plotly.subplots import make_subplots

# Convert the Time column to a string format for plotting
weather_data_str_time = race.weather_data.copy()
weather_data_str_time['Time_str'] = weather_data_str_time['Time'].apply(lambda x: str(x).split(' ')[-1]) # Extract HH:MM:SS


# Create subplots with multiple y-axes
weather_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for each weather metric
weather_fig.add_trace(
    go.Scatter(x=weather_data_str_time['Time_str'], y=weather_data_str_time['AirTemp'], name='Air Temp'),
    secondary_y=False,
)

weather_fig.add_trace(
    go.Scatter(x=weather_data_str_time['Time_str'], y=weather_data_str_time['TrackTemp'], name='Track Temp'),
    secondary_y=False,
)

weather_fig.add_trace(
    go.Scatter(x=weather_data_str_time['Time_str'], y=weather_data_str_time['Humidity'], name='Humidity'),
    secondary_y=True,
)

weather_fig.add_trace(
    go.Scatter(x=weather_data_str_time['Time_str'], y=weather_data_str_time['Pressure'], name='Pressure'),
    secondary_y=True,
)

weather_fig.add_trace(
    go.Scatter(x=weather_data_str_time['Time_str'], y=weather_data_str_time['WindSpeed'], name='Wind Speed'),
    secondary_y=True,
)

# Update layout to ensure y-axis range is set
weather_fig.update_layout(
    title='Weather Data During the Race',
    xaxis_title='Time', # Keep Time as x-axis title
    legend_title='Metric'
)

weather_fig.update_yaxes(title_text="Temperature (°C)", secondary_y=False)
weather_fig.update_yaxes(title_text="Value", secondary_y=True)

# Get the y-axis range after adding traces and updating layout
y_range_primary = weather_fig.layout.yaxis.range


# Add shading to indicate rain
# Find the start and end times of consecutive rain periods
rain_periods_str_time = weather_data_str_time[weather_data_str_time['Rainfall'] == True].copy()
if not rain_periods_str_time.empty:
    # Identify consecutive rain periods based on original Time difference
    rain_periods_str_time['rain_group'] = (rain_periods_str_time['Time'].diff() > pd.Timedelta(seconds=65)).cumsum()
    for group_id, group_df in rain_periods_str_time.groupby('rain_group'):
        start_time_str = group_df['Time_str'].min()
        end_time_str = group_df['Time_str'].max()

        # Use a default range if the actual range is still None
        y0_val = y_range_primary[0] if y_range_primary is not None else 0
        y1_val = y_range_primary[1] if y_range_primary is not None else 100 # Assuming a reasonable default max value


        weather_fig.add_shape(
            type="rect",
            x0=start_time_str,
            y0=y0_val,  # Start at the bottom of the primary y-axis
            x1=end_time_str,
            y1=y1_val,  # End at the top of the primary y-axis
            fillcolor="blue",
            opacity=0.2,
            layer="below",
            line_width=0,
        )
    # Add a single legend entry for "Rain"
    weather_fig.add_trace(go.Scatter(
        x=[None], y=[None], # Invisible trace
        mode='markers',
        marker=dict(size=10, color="blue", opacity=0.5),
        legendgroup='Rain',
        showlegend=True,
        name='Rain'
    ))


weather_fig.show()

## Analysing Tyre-Strategy by driver

In the next step we'll check out the tyre strategies which were choosen by the drivers and their teams. Even though these stongly depend on what happened within the race, I think it can be helpfull to make some basic guess.

In [ ]:
drivers = race.laps['Driver'].unique()

In [ ]:
stints = race.laps[['Driver', 'Stint', 'Compound', 'LapNumber']]
stints = stints.groupby(['Driver', 'Stint', 'Compound']).count().reset_index()
stints = stints.rename(columns={'LapNumber': 'LapCount'})

In [ ]:
import plotly.graph_objects as go

# Create a dictionary to map compound names to colors
compound_colors = {
    'SOFT': 'red',
    'MEDIUM': 'yellow',
    'HARD': 'white',
    'INTERMEDIATE': 'green',
    'WET': 'blue'
}

fig = go.Figure()

# Create a set to keep track of added compounds to avoid duplicate legend entries
added_compounds = set()

for driver in drivers:
    driver_stints = stints.loc[stints["Driver"] == driver].sort_values(by='Stint') # Sort by stint to ensure correct stacking

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        compound = row["Compound"]
        color = compound_colors.get(compound.upper(), 'gray') # Get color from dictionary, default to gray

        # Determine whether to show the legend entry for this compound
        show_legend_entry = False
        if compound not in added_compounds:
            added_compounds.add(compound)
            show_legend_entry = True

        fig.add_trace(go.Bar(
            y=[driver],
            x=[row["LapCount"]],
            name=compound,
            orientation='h',
            marker=dict(
                color=color,
                line=dict(color='white', width=2)
            ),
            base=previous_stint_end,
            customdata=[compound], # Store compound name in customdata for hover
            hovertemplate='Driver: %{y}<br>Compound: %{customdata}<br>Laps: %{x}<extra></extra>', # Custom hover text
            showlegend=show_legend_entry
        ))

        previous_stint_end += row["LapCount"]

fig.update_layout(
    title='Tyre Strategy per Driver',
    xaxis_title='Lap Number',
    yaxis_title='Driver',
    barmode='stack',
    legend_title='Compound',
    yaxis=dict(autorange="reversed"), # Invert y-axis
    height=800 # Adjust height for better readability
)

# Define track status colors
track_status_colors = {
    "AllClear": "green",
    "Yellow": "yellow",
    "Red": "red",
    "SCDeployed": "purple",
    "VSCDeployed": "violet",
    "VSCEnding": "orange"
}

# Add vertical lines for track status changes
for index, row in track_status_changes.iterrows():
    fig.add_vline(
        x=row['LapNumber'],
        line_width=1,
        line_dash="dash",
        line_color=track_status_colors[row['Message']], # Set line color based on track status
    )

    # Add hover information using a scatter trace with invisible markers
    fig.add_trace(go.Scatter(
        x=[row['LapNumber']],
        y=[0], # Place the marker at the bottom of the plot (arbitrary y-value)
        mode='markers',
        hoverinfo='text',
        text=f"Track Status: {row['Message']}, Lap {row['LapNumber']}",
        showlegend=False # Hide this trace from the legend
      ))


fig.show()

As you can see by hovering above the vertical dash-lines there were three situations that caused a red flag (lap 11,31, 43). All teams decided to take the red flag in lap 31 to make a pit stop and change tires. Because of the beginning rain in lap 26, 5 drivers took the decision to use the wet compound. Respectivly the wrong decision as they lost time and only Tsunoda and Lawson finished within the top ten.

To understand the effect of this wrong strategy we will try  in the next steps to evaluate the pit stop time of these drivers and moreover comparing the average laptime with the Intermediate Compound Tyres and the Wet Compound Tires.

In [ ]:
# drivers that used wet compound
drivers = ['TSU', 'LAW', 'PER', 'ZHO', 'HUL']

for driver in drivers:
  data = race.laps[race.laps['Driver'] == driver]

## Analysing Laptime Performence per Driver

In [ ]:
driver_laps_by_compound = {}

for driver in drivers:
    driver_laps = race.laps[race.laps['Driver'] == driver].copy()
    wet_intermediate_laps = driver_laps[driver_laps['Compound'].isin(['WET', 'INTERMEDIATE'])].copy()
    driver_laps_by_compound[f'{driver}_wet_intermediate_laps'] = wet_intermediate_laps

# Display the keys of the dictionary to show the created dataframes
print(driver_laps_by_compound.keys())

dict_keys(['TSU_wet_intermediate_laps', 'LAW_wet_intermediate_laps', 'PER_wet_intermediate_laps', 'ZHO_wet_intermediate_laps', 'HUL_wet_intermediate_laps'])


In [ ]:
# Access race control messages
race_control_messages = race.race_control_messages

# Filter for relevant track status changes
track_status_changes = race_control_messages[
    race_control_messages['Category'].isin(['Track Status', 'SafetyCar', 'RedFlag', 'VSC'])
].copy()

# Display the filtered track status changes before finding closest laps
print("Filtered track status changes before finding closest laps:")
display(track_status_changes)

# Get the session start time
session_start_time = race.session_start_time

# Convert race.laps['Time'] (timedelta from start) to datetime objects by adding session start time
race.laps['DateTime'] = session_start_time + race.laps['Time']

# Ensure 'Time' in track_status_changes is datetime
if not pd.api.types.is_datetime64_any_dtype(track_status_changes['Time']):
    track_status_changes['Time'] = pd.to_datetime(track_status_changes['Time'])

# Convert Time to lap number by finding the closest lap time
lap_numbers = []
for index, row in track_status_changes.iterrows():
    # Calculate absolute time differences between the track status change time and all lap datetimes
    # Ensure both are datetime before subtraction
    if pd.api.types.is_datetime64_any_dtype(race.laps['DateTime']) and pd.api.types.is_datetime64_any_dtype(row['Time']):
        time_diffs = np.abs(race.laps['DateTime'] - row['Time'])

        # Find the index of the minimum time difference
        closest_lap_index = time_diffs.idxmin()

        # Get the LapNumber corresponding to the closest time
        lap_numbers.append(race.laps.loc[closest_lap_index, 'LapNumber'])
    else:
        # If types are not compatible, append None
        lap_numbers.append(None)


# Add the determined lap numbers to the track_status_changes DataFrame
track_status_changes['LapNumber'] = lap_numbers

# Remove any rows where a corresponding lap number could not be found
track_status_changes = track_status_changes.dropna(subset=['LapNumber'])

# Convert the 'LapNumber' column to integer type
track_status_changes['LapNumber'] = track_status_changes['LapNumber'].astype(int)

# Display the resulting DataFrame containing the processed track status changes
display(track_status_changes)

Filtered track status changes before finding closest laps:


,Time,Category,Message,Status,Flag,Scope,Sector,RacingNumber,Lap
45,2024-11-03 16:28:21,SafetyCar,VIRTUAL SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,28
48,2024-11-03 16:29:50,SafetyCar,VIRTUAL SAFETY CAR ENDING,ENDING,None,None,NaN,None,28
54,2024-11-03 16:32:53,SafetyCar,SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,30
86,2024-11-03 17:13:14,SafetyCar,SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,39
88,2024-11-03 17:18:04,SafetyCar,SAFETY CAR IN THIS LAP,IN THIS LAP,None,None,NaN,None,42


,Time,Category,Message,Status,Flag,Scope,Sector,RacingNumber,Lap,LapNumber


In [ ]:
# Access race control messages
race_control_messages = race.race_control_messages

# Filter for relevant track status changes
track_status_changes = race_control_messages[
    race_control_messages['Category'].isin(['Track Status', 'SafetyCar', 'RedFlag', 'VSC'])
].copy()

# Use the existing 'Lap' column as the LapNumber
track_status_changes['LapNumber'] = track_status_changes['Lap']

# Remove any rows where LapNumber is missing (should not happen based on filtering, but as a safeguard)
track_status_changes = track_status_changes.dropna(subset=['LapNumber'])

# Convert the 'LapNumber' column to integer type
track_status_changes['LapNumber'] = track_status_changes['LapNumber'].astype(int)

# Display the resulting DataFrame containing the processed track status changes
display(track_status_changes)

,Time,Category,Message,Status,Flag,Scope,Sector,RacingNumber,Lap,LapNumber
45,2024-11-03 16:28:21,SafetyCar,VIRTUAL SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,28,28
48,2024-11-03 16:29:50,SafetyCar,VIRTUAL SAFETY CAR ENDING,ENDING,None,None,NaN,None,28,28
54,2024-11-03 16:32:53,SafetyCar,SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,30,30
86,2024-11-03 17:13:14,SafetyCar,SAFETY CAR DEPLOYED,DEPLOYED,None,None,NaN,None,39,39
88,2024-11-03 17:18:04,SafetyCar,SAFETY CAR IN THIS LAP,IN THIS LAP,None,None,NaN,None,42,42


In [ ]:
# Define track status colors if not already defined
track_status_colors = {
    "AllClear": "green",
    "Yellow": "yellow",
    "Red": "red",
    "SCDeployed": "purple",
    "VSCDeployed": "violet",
    "VSCEnding": "orange"
}

# Define compound colors if not already defined
compound_colors = {
    'SOFT': 'red',
    'MEDIUM': 'yellow',
    'HARD': 'white',
    'INTERMEDIATE': 'green',
    'WET': 'blue'
}

for driver in drivers:
    # Access the filtered lap data for the current driver
    driver_df = driver_laps_by_compound.get(f'{driver}_wet_intermediate_laps')

    if driver_df is not None and not driver_df.empty:
        # Convert LapTime to seconds for plotting
        driver_df['LapTimeSeconds'] = driver_df['LapTime'].dt.total_seconds()

        # Create a bar chart for the current driver's lap times
        fig = px.bar(driver_df,
                     x='LapNumber',
                     y='LapTimeSeconds',
                     color='Compound',
                     title=f'{driver} Lap Times by Compound',
                     labels={'LapTimeSeconds': 'Lap Time (seconds)'},
                     color_discrete_map=compound_colors) # Use the defined compound_colors

        # Add vertical lines for track status changes
        for index, row in track_status_changes.iterrows():
            fig.add_vline(
                x=row['LapNumber'],
                line_width=1,
                line_dash="dash",
                line_color=track_status_colors.get(row['Message'], 'gray'), # Get color from dictionary, default to gray
                annotation_text=f"{row['Message']} (Lap {row['LapNumber']})", # Add text annotation
                annotation_position="top" # Position the annotation at the top
            )


        # Update layout for better readability
        fig.update_layout(
            xaxis_title='Lap Number',
            yaxis_title='Lap Time (seconds)',
            showlegend=True # Ensure legend is shown
        )

        # Display the plot for the current driver
        fig.show()
    else:
        print(f"No wet or intermediate laps found for driver {driver}")